In [ ]:
# package for managing experiments
from sad2_final_project.analysis import BooleanNetworkExperiment
# system packages
import os
from pathlib import Path
import numpy as np

cwd=Path.cwd()
if cwd.name == "notebooks":
    os.chdir(cwd.parent) 
print(os.getcwd())

## create paths 
DATA_PATH = Path('data')
MODELS_PATH = Path("download_models")
MODEL_NAME = "model_id_37"
## create directories
!mkdir {DATA_PATH}
!mkdir {DATA_PATH / MODEL_NAME}
!mkdir {DATA_PATH / MODEL_NAME / "results"}
!mkdir {DATA_PATH / MODEL_NAME / "datasets"}
!mkdir {DATA_PATH / MODEL_NAME / "bn_ground_truth"}
!mkdir {DATA_PATH / MODEL_NAME / "datasets_bnfinder"}


/home/szot/repos/sad/SAD2_final_project
mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘data/model_id_37’: File exists
mkdir: cannot create directory ‘data/model_id_37/results’: File exists
mkdir: cannot create directory ‘data/model_id_37/datasets’: File exists
mkdir: cannot create directory ‘data/model_id_37/bn_ground_truth’: File exists
mkdir: cannot create directory ‘data/model_id_37/datasets_bnfinder’: File exists


In [6]:
from sad2_final_project.boolean_bn.bn import BN
from sad2_final_project.boolean_bn.bn_sampling import simulate_trajectories_to_csv
from sad2_final_project.bnfinder import manager_bnfinder

def run_single_condition(bn, data_path, experiment_name) -> bool:
    """
    Executes ONE experimental condition.
    This function is process-safe.
    """


    if (bn.mode == 'asynchronous'):
        sampling_frequency = 4
        k = 10 # TODO
    else:
        sampling_frequency = 1
        k = 7
    trajectory_length = int(round(bn.num_nodes * 0.8))
    n_trajectories = int(round((bn.num_nodes * k)/(trajectory_length)))


    # ---------- paths ----------
    gt_path = data_path / experiment_name / "bn_ground_truth" / f"{experiment_name}.csv"
    dataset_path = data_path / experiment_name / "datasets" / f"{experiment_name}.csv"
    bnf_dataset_path = data_path / experiment_name / "datasets_bnfinder" / f"{experiment_name}.txt"
    metrics_path = data_path / experiment_name / "results" / f"{experiment_name}_metrics.csv"

    # ---------- 2. save ground truth ----------
    bn.save_ground_truth(gt_path)

    # ---------- 3. generate data ----------
    # oznaczamy czy wygenerowany dataset powiódł sie sukcesem (potrzebne do zliczania sukcesów i jeżeli brak sukcesu nie próbujemy uruchamiać bnfindera bo brak pliku)
    success, ratio = simulate_trajectories_to_csv(
        bn_instance=bn,
        num_trajectories=n_trajectories,
        output_file=dataset_path,
        sampling_frequency=sampling_frequency,
        trajectory_length=trajectory_length,
        target_attractor_ratio = 0.5,
        tolerance = 0.5
    )

    # if not success:
    #     print(f"[INFO] Dataset {dataset_path} NOT created (requirements not met). Skipping inference.")
    #     return False

    # ---------- 4. inference ----------
    manager_bnfinder(
        ## file paths
        dataset_path=dataset_path,
        ground_truth_path=gt_path,
        trained_model_name= data_path / experiment_name / "results" / experiment_name,
        bnf_file_path=bnf_dataset_path,
        metrics_file=metrics_path,
        ## model parameters
        #score_functions=bn.score_functions,
        ## analysis settings
        #analysis_metrics=self.analysis_metrics,
        #analysis_score_functions=self.analysis_score_functions,
        dataset_succeeded=success,
        attractor_ratio=ratio,
    )

    return True

In [7]:
from sad2_final_project.boolean_bn import load_bnet_to_BN

bn = load_bnet_to_BN(MODELS_PATH / MODEL_NAME / "model.bnet")
if run_single_condition(bn, DATA_PATH, MODEL_NAME):
    print(f"Experiment {MODEL_NAME} completed successfully.")
else:
    print(f"Experiment {MODEL_NAME} failed.")

Loaded 16 nodes. Mapping:
  x0: v_AKT
  x1: v_EGF
  x2: v_ERBB1
  x3: v_ERBB2
  x4: v_ERBB3
  x5: v_ERK1_2
  x6: v_HRG
  x7: v_MEK1_2
  x8: v_PDK1
  x9: v_PKCa
  x10: v_PLCg
  x11: v_erlotinib
  x12: v_mTOR
  x13: v_p70S6K
  x14: v_pertuzumab
  x15: v_trastuzumab
eee
Experiment model_id_37 completed successfully.
